<a href="https://colab.research.google.com/github/samin9796/arg2keypoint/blob/main/Copy_of_Few_shot_learning_with_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Few shot text generation with T5 Transformer**

Author: Ramsri Goutham Golla

Linkedin : https://www.linkedin.com/in/ramsrig/

Twitter: https://twitter.com/ramsri_goutham

## 1. Install libraries

In [ ]:
!pip install transformers==2.9.0

     |████████████████████████████████| 635 kB 5.5 MB/s 
     |████████████████████████████████| 5.6 MB 40.6 MB/s 
     |████████████████████████████████| 1.2 MB 36.4 MB/s 
     |████████████████████████████████| 880 kB 43.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=2bc2618f45f6b4913b8a59e17ab7e4a2bf7c29a9cb9956742cf5402ed0b8bc76
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [ ]:
# Check we have a GPU and check the memory size of the GUP
!nvidia-smi

Tue May 10 11:06:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 2. Prepare Model

In [ ]:

import random
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model1 = T5ForConditionalGeneration.from_pretrained('t5-base')


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
# optimizer
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in t5_model1.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in t5_model1.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-4, eps=1e-8)



In [ ]:
train_df = pd.read_csv("train.csv")
dev_df = pd.read_csv("dev.csv")

In [ ]:
arg_keypoint_tuples=[]
for i in train_df.index:
  arg = train_df['argument'][i]
  keypoint = train_df['key_point'][i]
  if train_df['label'][i]==1:
    label ="matched"
  else:
    label="unmatched"
  if arg[-1] != '.':
    modified_arg = arg + '.'
    train_df.at[i, 'argument'] = modified_arg
  else:
    modified_arg = arg 
    train_df.at[i, 'argument'] = modified_arg
  
  arg_keypoint_tuples.append((arg, keypoint, label))

In [ ]:
import random

random.shuffle(arg_keypoint_tuples)
train_set=arg_keypoint_tuples[:100]


In [ ]:
print(train_set[0])

('child actors cannot be banned since a typical family would never be able to be depicted on film or television.', "Acting may be the child's dream", 'unmatched')


In [ ]:
print(type(arg_keypoint_tuples))

<class 'list'>


In [ ]:
# # dataset preparation

# arg_keypoint_tuples = [
#                                ("a person should have the right to be able to choose if they want to live or die.","Assisted suicide reduces suffering", "unmatched"),
#                                ("assisted suicide allows one to end terminal pain and suffering and should be allowed to continue.","Assisted suicide gives dignity to the person that wants to commit it", "unmatched"),
#                                ("assisted suicide allows people who have terrible health conditions to die with dignity. if assisted suicide is a crime they may have to endure many more years living in agony.","People should have the freedom to choose to end their life", "unmatched"),
#                                ("assisted suicide allows terminally ill people to die with dignity and should not be criminalized.","People should have the freedom to choose to end their life", "unmatched"),
#                                ("assisted suicide allows terminally ill people to die with dignity and should not be criminalized.","The terminally ill would benefit from assisted suicide", "unmatched"),
#                                ("assisted suicide helps terminally ill people end their suffering.","Assisted suicide gives dignity to the person that wants to commit it", "unmatched"),
#                                ("Assisted suicide helps those who are in pain due to a devastating disease end their own lives on their own terms.","Assisted suicide gives dignity to the person that wants to commit it", "unmatched"),
#                                ("assisted suicide is a human right and would spare the terminally i'll pain.","Assisted suicide gives dignity to the person that wants to commit it", "unmatched"),
#                                ("helping someone commit suicide is like killing them yourself.","Assisted suicide is akin to killing someone","matched"),
#                                ("helping someone kill themself is murder", "Assisted suicide is akin to killing someone", "matched"),
#                                ("the vow of celibacy is unnatural and causes harm.","Celibacy is unhealthy/unnatural","matched"),
#                                ("marriage is a sacred bond that is highly regarded in various religions.","Marriage is important for people, either generally or because of religious/traditional reasons","matched"),
#                                ("abolishing intellectual property rights would damage the economy since there would be less incentive to bring new and innovative products to market","Intellectual property rights incentivize investment in developing new products","matched")

# ]

## 3. Train Loop

In [ ]:
t5_model1.train()

epochs = 5

for epoch in range(epochs):
  print ("epoch ",epoch)
  for input1, input2, output in train_set:
    input_sent ="The argument:" +input1+ " The keypoint: "+input2+ "is </s>"
    ouput_sent = output+" </s>"

    tokenized_inp = tokenizer.encode_plus(input_sent,  max_length=96, pad_to_max_length=True,return_tensors="pt")
    tokenized_output = tokenizer.encode_plus(ouput_sent, max_length=96, pad_to_max_length=True,return_tensors="pt")


    input_ids  = tokenized_inp["input_ids"]
    attention_mask = tokenized_inp["attention_mask"]

    lm_labels= tokenized_output["input_ids"]
    decoder_attention_mask=  tokenized_output["attention_mask"]


    # the forward function automatically creates the correct decoder_input_ids
    output = t5_model1(input_ids=input_ids, lm_labels=lm_labels,decoder_attention_mask=decoder_attention_mask,attention_mask=attention_mask)
    loss = output[0]

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()




epoch  0
epoch  1
epoch  2
epoch  3
epoch  4


In [ ]:
test_df = pd.read_csv("test.csv")


In [ ]:
arg_keypoint_test=[]
test_ref = []
for i in test_df.index:
  arg = test_df['argument'][i]
  keypoint = test_df['key_point'][i]
  if test_df['label'][i]==1:
    label ="matched"
  else:
    label="unmatched"
  if arg[-1] != '.':
    modified_arg = arg + '.'
    test_df.at[i, 'argument'] = modified_arg
  else:
    modified_arg = arg 
    test_df.at[i, 'argument'] = modified_arg
  output_sent ="The argument:" +arg+ " The keypoint: " +keypoint+ " is </s>"
  test_ref_single= "The argument:" +arg+ " The keypoint: " +keypoint+ " is " + label + "</s>"
  arg_keypoint_test.append(output_sent)
  test_ref.append(test_ref_single)

In [ ]:


test_set=arg_keypoint_test[:50]

In [ ]:
print(arg_keypoint_test[1])

The argument:everyone should have the right to their own freedom of speech, even if it means sending that message via a flag burning. The keypoint: flag burning is justified due to freedom of expression is </s>


## 4. Test model

In [ ]:
cnt = 0
for i in test_ref:
  print(i)
  cnt+=1
  if cnt==1:
    break

for i in test_output:
  print(i)
  if cnt==1:
    break

The argument:a large majority of the world believes in god in some form so adopting atheism is not going with what the majority believes. The keypoint: Atheism lacks morality is unmatched</s>
The argument:flag burning is disrespected to anyone who has ever faught for their country. The keypoint: Flags are important symbols that should be respected is </s>unmatched


In [ ]:
print(len(arg_keypoint_test))

3617


In [ ]:
test_output=[]
for i in test_set:
  test_sent = i
 
  test_tokenized = tokenizer.encode_plus(test_sent, return_tensors="pt")

  test_input_ids  = test_tokenized["input_ids"]
  test_attention_mask = test_tokenized["attention_mask"]

  t5_model1.eval()
  beam_outputs = t5_model1.generate(
      input_ids=test_input_ids,attention_mask=test_attention_mask,
      max_length=64,
      early_stopping=True,
      num_beams=10,
      num_return_sequences=1,
      no_repeat_ngram_size=2
  )

  for beam_output in beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      # print (sent)
      test_output_single=i+sent
      test_output.append(test_output_single)


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


In [ ]:
print(test_output[1])
print(test_ref[1])

The argument:if someone can still work they should be allowed. it will cause less strain on social security. The keypoint: A mandatory retirement is not fair/discriminatory is </s>unmatched
The argument:a large majority of the world believes in god in some form so adopting atheism is not going with what the majority believes. The keypoint: People should choose for themselves whether Atheism should be adopted is unmatched</s>


In [ ]:
print(test_ref[0],test_output[0])

NameError: ignored

In [ ]:
test_sent = 'The argument: the vow of celibacy is unnatural and causes harm. The keypoint: Celibacy is unhealthy/unnatural is </s>'
test_tokenized = tokenizer.encode_plus(test_sent, return_tensors="pt")

test_input_ids  = test_tokenized["input_ids"]
test_attention_mask = test_tokenized["attention_mask"]

t5_model1.eval()
beam_outputs = t5_model1.generate(
    input_ids=test_input_ids,attention_mask=test_attention_mask,
    max_length=64,
    early_stopping=True,
    num_beams=10,
    num_return_sequences=1,
    no_repeat_ngram_size=2
)

for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print (sent)

/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


unmatched


In [ ]:
!nvidia-smi